In [ ]:
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.errors as errors
import azure.cosmos.http_constants as http_constants
import json
import os
import csv

url = 'https://8d4abf2e-0ee0-4-231-b9ee.documents.azure.com:443/'
key = 'is5y6iEm1oTpKeu18XppWBSzgvX231mBVwyf9JgU8jgFjMG21UVD7FbjmG7GmD9lLnrv3sBXduqmNzaOFey3uw=='
client = cosmos_client.CosmosClient(url, {'masterKey': key})

In [ ]:
databaes_name = 'AccentureData'
database = client.get_database_client(databaes_name)
query = "SELECT MAX(databaes_nam) FROM databaes_name"
properties = database.query_containers(query=query,enable_cross_partition_query=True)


In [ ]:
import pandas as pd

container_name = 'ODL_ORDER_ITEM'
container = database.get_container_client(container_name)

query = "SELECT * FROM container_name WHERE container_name.id IN ('1','2','3')"

container_dict = container.query_items(
        query=query,enable_cross_partition_query=True)

In [ ]:
containers = [container['id'] for container in database.list_containers()]
containers

In [ ]:
for container_id in containers:
    container = database.get_container_client(container_id)
    iter_items = container.read_all_items(max_item_count=1000)
    keys = container.read_item("0","0").keys()
    
    with open(f'{container_id}.csv', 'w', newline='') as output_file:
            dict_writer = csv.DictWriter(output_file, keys)
            dict_writer.writeheader()
            dict_writer.writerows(iter_items)

In [ ]:
for container_id in containers:
    container = database.get_container_client(container_id)
    iter_items = container.read_all_items(max_item_count=1000)
    keys = container.read_item
    

In [66]:
import pandas as pd
pd.set_option('display.max_rows', 300)

read_none = ["None"]
order_item_df = pd.read_csv('/home/becode2/business_analysis/ODL_ORDER_ITEM.csv', na_values = read_none)
orderables_df = pd.read_csv('/home/becode2/business_analysis/ODL_ORDERABLES.csv', na_values = read_none)
order_df = pd.read_csv('/home/becode2/business_analysis/ODL_ORDER.csv', na_values = read_none)
allergy_customer_df = pd.read_csv('/home/becode2/business_analysis/ODL_ALLERGY_CUSTOMER.csv', na_values = read_none)
allergy_df = pd.read_csv('/home/becode2/business_analysis/ODL_ALLERGY.csv', na_values = read_none)
restaurant_df = pd.read_csv('/home/becode2/business_analysis/ODL_RESTAURANT.csv', na_values = read_none)
orders_value_df = pd.read_csv('/home/becode2/business_analysis/orders_value.csv', na_values = read_none)
restaurant_revenue_df = pd.read_csv('/home/becode2/business_analysis/restaurant_revenue.csv', na_values = read_none)

In [ ]:
from geopy.geocoders import Nominatim 
geolocator = Nominatim(user_agent="geoapiExercises")

df = restaurant_df #dataframe is imported from csv before

nan_rest = df[df['city'].isna()] #streets with empty cities

"""first i tried to see if geocoder will find the same street in both cities"""
list_ny_street= []
for s in nan_rest['street']:
    try:
        place_ny = geolocator.geocode(f'{s}, New York')        
        if place_ny != None:
            list_ny_street.append({'street':s, 'city':'New York', 'result':place_ny})#this ditc is only so i can see the results and comapre street with place
    except:# the script will raise an error if a street is not found in NY so i skip that
        pass
    
list_sf_street= []
for s in nan_rest['street']:
    try:
        place_sf = geolocator.geocode(f'{s}, San Francisco')        
        if place_sf != None:
            list_sf_street.append({'street':s, 'city':'San Francisco', 'result':place_sf})
    except:
        pass
    
# After looking at both lists i realised that SF list was more accurate since NY list was looking at the state of NY

sf_street = []
for i in list_sf_street:#created q new list with the streets thqt belong in SF
    sf_street.append(i['street'])
    
for d1 in list_ny_street:#remove SF values that are still in NY list
    for d2 in list_sf_street:
        if d1['street'] == d2['street']:
            list_ny_street.remove(d1)
            
ny_street = []#created q new list with the final list of NY streets
for i in list_ny_street:
    ny_street.append(i['street'])
    
#added street values to the datframe

for i in ny_street:
    df.loc[df.street == i, ['city']] = "New York"

for i in sf_street:
    df.loc[df.street == i, ['city']] = "San Francisco"
    
ny_street_missing = ['Emmet Ct','Tenny Pl','Legion Memorial Square' ]
sf_street_missing = ['Doublerock St', 'Winn Way - Golden Gate National Recreation Area','S Hughes Ln - Golden Gate National Recreation Area','Harrison Blvd - Golden Gate National Recreation Area']

for i in ny_street_missing:
    df.loc[df.street == i, ['city']] = "New York"
    
for i in sf_street_missing:
    df.loc[df.street == i, ['city']] = "San Francisco"
    
df.to_csv('ODL_RESTAURANT.csv',mode='w+' ) #overwrite the csv

In [41]:

order_item_df_value = order_item_df.join(orderables_df[['price','name']], on='orderable_id')
order_item_df_value["order_value"] = order_item_df_value['price'].mul(order_item_df_value['amount']) #order item with total value per item
order_df[['creation_date_only', 'creation_time']] = order_df['creation_date'].str.split(' ', 1, expand=True)
pd.to_datetime(order_df['creation_date_only'],format="%x")
orders_value = order_item_df_value.groupby(['order_id'])['order_value'].sum().sort_values(ascending=False).to_frame()#total value per order
order_value_df = order_df.join(orders_value['order_value'], on='id')
order_value_df['month_year']= pd.to_datetime(order_value_df['creation_date_only']).dt.strftime('%y/%m')
order_value_df['month']= pd.to_datetime(order_value_df['creation_date_only']).dt.strftime('%m')
restaurant_city = restaurant_df.set_index('data_id')
restaurant_city.index.name = None
order_value_df 


,restaurant_id,creation_date,customer_id,id,creation_date_only,creation_time,order_value,month_year,month
0,91,8/25/17 18:49,2858,17265,8/25/17,18:49,131.1,17/08,08
1,91,12/20/17 11:39,10055,44615,12/20/17,11:39,155.5,17/12,12
2,91,11/23/18 10:12,13703,137232,11/23/18,10:12,247.9,18/11,11
3,91,11/2/18 17:33,14698,131091,11/2/18,17:33,146.9,18/11,11
4,91,7/24/18 9:49,14299,102153,7/24/18,9:49,128.4,18/07,07
...,...,...,...,...,...,...,...,...,...
178719,404,2/8/19 18:23,19527,161413,2/8/19,18:23,92.1,19/02,02
178720,412,3/15/19 14:46,20362,173165,3/15/19,14:46,176.9,19/03,03
178721,402,1/3/19 10:19,9193,149327,1/3/19,10:19,110.7,19/01,01
178722,400,2/4/19 18:05,15050,160047,2/4/19,18:05,57.6,19/02,02


In [62]:
orders_value_df = order_value_df.join(restaurant_city[['city']], on='restaurant_id')
orders_value_df.to_csv('orders_value.csv',mode='w+' ) #total value per order with date,time, month and, month_year

In [28]:
# Info company page

def ny_clv():
    ny_avg_order_value = order_value__city_df.loc[order_value__city_df['city'] == 'New York'].groupby(['customer_id'])['order_value'].sum().mean()
    ny_avg_repeat_purchase = order_value__city_df.loc[order_value__city_df['city'] == 'New York'].groupby(['customer_id'])['order_value'].count().mean()
    ny_clv = ny_avg_order_value * ny_avg_repeat_purchase
    return ny_clv

def sf_clv():
    sf_avg_order_value = order_value__city_df.loc[order_value__city_df['city'] == 'San Francisco'].groupby(['customer_id'])['order_value'].sum().mean()
    sf_avg_repeat_purchase = order_value__city_df.loc[order_value__city_df['city'] == 'San Francisco'].groupby(['customer_id'])['order_value'].count().mean()
    sf_clv = sf_avg_order_value * sf_avg_repeat_purchase
    return ny_clv()

def total_clv():
    return (ny_clv() + sf_clv()) / 2

#qvg customer repeat purchase
def sf_crp():
    sf_crp = order_df.loc[order_value__city_df['city'] == 'San Francisco'].groupby(['customer_id'])['restaurant_id'].count().sort_values(ascending=True).to_frame()['restaurant_id'].mean()
    return sf_crp

def ny_crp():
    ny_crp = order_df.loc[order_value__city_df['city'] == 'New York'].groupby(['customer_id'])['restaurant_id'].count().sort_values(ascending=True).to_frame()['restaurant_id'].mean()
    return ny_crp

def total_crp():
    return (sf_crp() + ny_crp()) / 2


,Unnamed: 0,restaurant_id,creation_date,customer_id,id,creation_date_only,creation_time,order_value,month_year,month,city
0,0,91,8/25/17 18:49,2858,17265,8/25/17,18:49,131.1,17/08,8,New York
1,1,91,12/20/17 11:39,10055,44615,12/20/17,11:39,155.5,17/12,12,New York
2,2,91,11/23/18 10:12,13703,137232,11/23/18,10:12,247.9,18/11,11,New York
3,3,91,11/2/18 17:33,14698,131091,11/2/18,17:33,146.9,18/11,11,New York
4,4,91,7/24/18 9:49,14299,102153,7/24/18,9:49,128.4,18/07,7,New York
...,...,...,...,...,...,...,...,...,...,...,...
178719,178719,404,2/8/19 18:23,19527,161413,2/8/19,18:23,92.1,19/02,2,New York
178720,178720,412,3/15/19 14:46,20362,173165,3/15/19,14:46,176.9,19/03,3,New York
178721,178721,402,1/3/19 10:19,9193,149327,1/3/19,10:19,110.7,19/01,1,San Francisco
178722,178722,400,2/4/19 18:05,15050,160047,2/4/19,18:05,57.6,19/02,2,New York


In [101]:
#info by client id
def clv_by_id(x):
    clv_by_id = order_value__city_df.loc[order_value__city_df['customer_id'] == x, 'order_value']
    return clv_by_id.sum()

def crp_by_id(x):
    crp_by_id = order_value__city_df.loc[order_value__city_df['customer_id'] == x, 'order_value']
    return crp_by_id.count()

def cab_by_id(x):
    cab_by_id = clv_by_id(x) / crp_by_id(x)
    return cab_by_id


140.3

In [72]:
#restaurant information page
x = 100 #Id of restaurant
y = '17/05' #year date

def popular_months(x):
    popular_months = order_value_df.loc[order_value_df['restaurant_id'] == x].groupby(['month'])['order_value'].sum().to_frame().sort_values('month')
    return popular_months.plot.bar(legend=True)

def revenue_by_month(x):
    revenue_by_month = order_value_df.loc[order_value_df['restaurant_id'] == x].groupby(['month_year'])['order_value'].sum().to_frame().sort_values('month_year')
    return revenue_by_month.plot(legend=True)

def revenue_one_month(x,y):
    revenue_by_month = order_value_df.loc[order_value_df['restaurant_id'] == x].groupby(['month_year'])['order_value'].sum().to_frame().sort_values('month_year')
    revenue_one_month = revenue_by_month.loc[y, 'order_value']
    return revenue_one_month

def restaurant_COG(x): #cost of goods
    cost_of_goods = restaurant_revenue_df.loc[restaurant_revenue['id'] == x, 'cost_of_goods_month']
    return cost_of_goods

def restaurant_COL(x): #cost of labor
    cost_of_labor = restaurant_revenue_df.loc[restaurant_revenue['id'] == x, 'cost_of_labor_month']
    return cost_of_labor

def restaurant_FC(x): #fixed costs
    fixed_cost = restaurant_revenue_df.loc[restaurant_revenue['id'] == x, 'fixed_costs']
    return fixed_cost

def restaurant_PNL_month(x, year_month):#profits and lost one month
    revenue_by_month = order_value_df.loc[order_value_df['restaurant_id'] == x].groupby(['month_year'])['order_value'].sum().to_frame().sort_values('month_year')
    restaurant_PNL_month = revenue_by_month.loc[year_month, 'order_value'] - ((revenue_by_month.loc[year_month, 'order_value'] * 0.15) + restaurant_COL(x) + restaurant_FC(x))
    return restaurant_PNL_month

def restaurant_revenue_overall(x): #cost of goods
    restaurant_revenue_overall = restaurant_revenue_df.loc[restaurant_revenue['id'] == x, 'revenue']
    return restaurant_revenue_overall.sum()


535.0

In [61]:
resto_value_df = order_value_df.groupby(['restaurant_id'])['order_value'].sum().sort_values(ascending=False)
resto_value_df = resto_value_df.to_frame()
restaurant_revenue = restaurant_df.join(resto_value_df['order_value'], on='id',)
restaurant_revenue = restaurant_revenue.rename(columns={"order_value":"revenue"})
restaurant_revenue # revenue per restaurant
#restaurand costs added to data frame
restaurant_revenue_df = restaurant_revenue[['id','opening_hours','city','creation_date','street','email','phone_number','name','revenue']]
restaurant_revenue_df['cost_of_goods'] = resto_value_df['order_value'] * 0.15
restaurant_revenue_df['cost_of_labor_month'] = (resto_value_df['order_value'].mean() * 0.41) / 12
restaurant_revenue_df['fixed_costs_month'] = (resto_value_df['order_value'].mean() * 0.36) / 12
restaurant_revenue_df['cost_of_labor_total'] = (resto_value_df['order_value'].mean() * 0.41) 
restaurant_revenue_df['fixed_costs_total'] = (resto_value_df['order_value'].mean() * 0.36) 
restaurant_revenue_df['total_PNL'] = restaurant_revenue_df['revenue'] - (restaurant_revenue_df['cost_of_goods'] + restaurant_revenue_df['fixed_costs_total'] + restaurant_revenue_df['cost_of_labor_total'])
restaurant_revenue_df.to_csv('restaurant_revenue_df.csv',mode='w+' )

/tmp/ipykernel_5842/2185613563.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurant_revenue_df['cost_of_goods'] = resto_value_df['order_value'] * 0.15
/tmp/ipykernel_5842/2185613563.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurant_revenue_df['cost_of_labor_month'] = (resto_value_df['order_value'].mean() * 0.41) / 12
/tmp/ipykernel_5842/2185613563.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 